In [ ]:
import regex as re
PAT=r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""


In [121]:
# from typing import Iterable,Iterator
# class Tokenizer:
#     def __init__(self, vocab: dict[int, bytes], merges: list[tuple[bytes, bytes]], special_tokens: list[str] | None = None):
#         self.vocab=vocab
#         self.merges=merges
#         self.special_tokens=special_tokens

#     def from_files(cls, vocab_filepath:str, merges_filepath:str, special_tokens: list[str] | None = None):
#         return

#     def encode(self,text: str) -> list[int]:
#         return
#     def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]: 
#         return

#     def decode(self, ids: list[int]) -> str:
#         return
    

In [94]:
import pickle
with open('tokenizer_TinyStories-train.pkl', 'rb') as f:
    data = pickle.load(f)
vocab = data['vocab']
merges = data['merges']

In [95]:
tok=Tokenizer(vocab,merges,["<|endoftext|>"])

In [132]:
s = "the cat ate<|endoftext|>"
voc = {
0: b' ',
1: b'a',
2: b'c',
3: b'e',
4: b'h',
5: b't',
6: b'th',
7: b' c',
8: b' a',
9: b'the',
10:b' at',
11:b'<|endoftext|>'
}
merges = [
(b't', b'h'),
(b' ', b'c'),
(b' ', b'a'),
(b'th', b'e'),
(b' a', b't')
]

第一步 预编码

In [123]:
from bpe_train import pre_tokenization
it = pre_tokenization(s,special_tokens=["<|endoftext|>"])
pre_tokens=[]
for i in it:
    pre_tokens.append(i.group())
pre_tokens

['the', ' cat', ' ate']

In [124]:
def split_token(token):
    splited=[]
    for c in token:
        splited.append(c.encode("utf-8"))
    return splited

In [125]:
def perform_merge(token,merges): # token='the'
    chars=split_token(token) # chars=b't',b'h',b'e'
    i=0
    while i<len(chars) - 1:
        # print(chars[i],chars[i+1])
        if (chars[i],chars[i+1]) in merges:
            combined = chars[i]+chars[i+1]
            # print(combined)
            chars = chars[:i] + [combined] + chars[i+2:]
        else:
            # print(f"{(chars[i],chars[i+1])} not in merges")
            i = i+1

    return chars

In [127]:
def map_merged_tokens_to_ids(merged_tokens,vocab):
    token_ids=[]
    for token in merged_tokens:
        for k,v in vocab.items():
            if token==v:
                token_ids.append(k)
    return token_ids

In [129]:
token_ids=[]
for token in pre_tokens:
    merged_tokens = perform_merge(token,merges)
    print(merged_tokens)
    ids = map_merged_tokens_to_ids(merged_tokens,voc)
    for i in ids:
        token_ids.append(i)
token_ids        

[b'the']
[b' c', b'a', b't']
[b' at', b'e']


[9, 7, 1, 5, 10, 3]

In [130]:
import regex as re

def pre_tokenization_with_specials(raw_text: str, 
                                   special_tokens=["<|endoftext|>"], 
                                   PAT=r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""):
    # 构造正则，将特殊符号作为优先匹配
    special_pattern = "|".join(re.escape(t) for t in special_tokens)
    combined_pattern = f"({special_pattern})|({PAT})"

    # 匹配：如果 group(1) 不为空，就是特殊符号；否则是普通词
    tokens_with_type = []
    for match in re.finditer(combined_pattern, raw_text):
        if match.group(1) is not None:  
            tokens_with_type.append((match.group(1), True))   # True 表示是特殊token
        elif match.group(2) is not None:
            tokens_with_type.append((match.group(2), False))  # False 表示普通token
    return tokens_with_type


In [131]:
class Tokenizer:
    def __init__(self, vocab: dict[int, bytes], merges: list[tuple[bytes, bytes]], special_tokens: list[str] | None = None):
        self.vocab = vocab
        self.merges = merges
        self.special_tokens = special_tokens or []

    def encode(self, text: str) -> list[int]:
        token_ids = []
        for token, is_special in pre_tokenization_with_specials(text, self.special_tokens):
            if is_special:
                # 直接查 vocab
                for k, v in self.vocab.items():
                    if v.decode() == token:
                        token_ids.append(k)
                        break
            else:
                merged_tokens = perform_merge(token, self.merges)
                ids = map_merged_tokens_to_ids(merged_tokens, self.vocab)
                token_ids.extend(ids)
        return token_ids


In [135]:
tok=Tokenizer(voc,merges,["<|endoftext|>"])

In [136]:
tok.encode(s)

[9, 7, 1, 5, 10, 3, 11]

In [1]:
from typing import Iterable, Iterator
import regex as re

def split_token(token: str):
    """将字符串 token 拆成 utf-8 bytes 列表"""
    return [c.encode("utf-8") for c in token]

def perform_merge(token: str, merges: list[tuple[bytes, bytes]]):
    """执行 BPE 合并"""
    chars = split_token(token)
    i = 0
    while i < len(chars) - 1:
        if (chars[i], chars[i+1]) in merges:
            combined = chars[i] + chars[i+1]
            chars = chars[:i] + [combined] + chars[i+2:]
        else:
            i += 1
    return chars

def map_merged_tokens_to_ids(merged_tokens: list[bytes], vocab: dict[int, bytes]):
    """将合并后的 bytes token 映射到 ID"""
    token_ids = []
    for token in merged_tokens:
        for k, v in vocab.items():
            if token == v:
                token_ids.append(k)
                break
    return token_ids

def pre_tokenization_with_specials(raw_text: str, 
                                   special_tokens: list[str], 
                                   PAT=r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""):
    """先识别特殊符号，再正则分词"""
    if not special_tokens:
        special_tokens = []
    special_pattern = "|".join(re.escape(t) for t in special_tokens)
    if special_pattern:
        combined_pattern = f"({special_pattern})|({PAT})"
    else:
        combined_pattern = f"({PAT})"

    tokens_with_type = []
    for match in re.finditer(combined_pattern, raw_text):
        if special_tokens and match.group(1) is not None:  
            tokens_with_type.append((match.group(1), True))   # 特殊token
        else:
            tok = match.group(2) if special_tokens else match.group(1)
            if tok:
                tokens_with_type.append((tok, False))         # 普通token
    return tokens_with_type


class Tokenizer:
    def __init__(self, vocab: dict[int, bytes], merges: list[tuple[bytes, bytes]], special_tokens: list[str] | None = None):
        self.vocab = vocab
        self.merges = merges
        self.special_tokens = special_tokens or []

    @classmethod
    def from_files(cls, vocab_filepath: str, merges_filepath: str, special_tokens: list[str] | None = None):
        # 加载 vocab
        vocab = {}
        with open(vocab_filepath, "rb") as vf:
            for i, line in enumerate(vf):
                vocab[i] = line.strip()

        # 加载 merges
        merges = []
        with open(merges_filepath, "rb") as mf:
            for line in mf:
                parts = line.strip().split()
                if len(parts) == 2:
                    merges.append((parts[0], parts[1]))

        return cls(vocab, merges, special_tokens)

    def encode(self, text: str) -> list[int]:
        token_ids = []
        for token, is_special in pre_tokenization_with_specials(text, self.special_tokens):
            if is_special:
                # 特殊符号直接映射
                for k, v in self.vocab.items():
                    if v.decode() == token:
                        token_ids.append(k)
                        break
            else:
                merged_tokens = perform_merge(token, self.merges)
                ids = map_merged_tokens_to_ids(merged_tokens, self.vocab)
                token_ids.extend(ids)
        return token_ids

    def encode_iterable(self, iterable: Iterable[str]) -> Iterator[int]: 
        for text in iterable:
            for token_id in self.encode(text):
                yield token_id

    def decode(self, ids: list[int]) -> str:
        tokens = [self.vocab[i].decode("utf-8") for i in ids]
        return "".join(tokens)


# ===== 测试 =====
if __name__ == "__main__":
    s = "the cat ate<|endoftext|>"

    voc = {
        0: b' ',
        1: b'a',
        2: b'c',
        3: b'e',
        4: b'h',
        5: b't',
        6: b'th',
        7: b' c',
        8: b' a',
        9: b'the',
        10: b' at',
        11: b'<|endoftext|>'
    }
    merges = [
        (b't', b'h'),
        (b' ', b'c'),
        (b' ', b'a'),
        (b'th', b'e'),
        (b' a', b't')
    ]

    tokenizer = Tokenizer(voc, merges, special_tokens=["<|endoftext|>"])
    ids = tokenizer.encode(s)
    print("Encoded IDs:", ids)
    print("Decoded Text:", tokenizer.decode(ids))


Encoded IDs: [9, 7, 1, 5, 10, 3, 11]
Decoded Text: the cat ate<|endoftext|>


In [2]:
import pickle
with open('tokenizer_TinyStories-train.pkl', 'rb') as f:
    data = pickle.load(f)
vocab = data['vocab']
merges = data['merges']
tokenizer = Tokenizer(vocab, merges, special_tokens=["<|endoftext|>"])

In [3]:
ids = tokenizer.encode("I want to eat a hamburger.")
print("Encoded IDs:", ids)
print("Decoded Text:", tokenizer.decode(ids))

Encoded IDs: [73, 271, 346, 265, 344, 296, 258, 2880, 8945, 114, 780, 114, 46]
Decoded Text: I want to eat a hamburger.
